<a href="https://colab.research.google.com/github/Rohaaq/Grade-12-CS-Project-AR-Brick-Breaker-/blob/main/Hand_Tracked_Environment_Explorer_GR12CS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Misc Block W/ Libraries

In [ ]:
# Installing Libraries in case stuff doesn't work
!pip install mediapipe
#!pip install google
#!pip install google.colab.output

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# Rohaan Haq
# Final CS project
# See Requirnments and Design doc. Will use handtracking to navigate a virtual enviorment
# Should be noted: I did initally refrence some of Adrian's mouse script whe starting this project

#Resource and To-Do List Page:
# https://docs.google.com/document/d/1-7RL2OQmJxO8wAYxSy-9iKUawdRrw2bgDn-mXoUEqMo/edit#

#Libraries

# Lib's for webcam
from IPython.display import Image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

#Lib's for Hand tracking
import mediapipe as mp
import numpy as np
import time
import cv2

# Grid Libraries
from google.colab import output
import random

#prints out info about parameters
#help(mp.solutions.hands.Hands)

#Module 1: Hand tracking & Output Movment


In [ ]:
# Helper Functions

# function to convert the JavaScript object into an OpenCV image
# Taken from this project (https://colab.research.google.com/drive/1xdjyBiY75MAVRSjgmiqI7pbRLn58VrbE?usp=sharing#scrollTo=xZk68RUQW3Xq)
def js_to_image(js_image):

  # decode base64 image
  image_bytes = b64decode(js_image.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img


# Function for handling the diplay of webcam (uses JavaScript)
# Also taken from above project. I did my best to comment but idk exactly what all this does.
# ALso has various helper functions
def video_stream():
  js = Javascript('''
    //Variables
      var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    // Removes frame, I think?
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    // I'm honestly not sure
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    // Initalizes window
    // Should prob revisit this for un-mirroring feed
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      // Feed caption
      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    // Condition for stoping stream
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      // Main logoic
      var preCreate = Date.now();
      // Creating window
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

#Gets the current video frame, and labels it
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# Functions and code for ASCII Maze

#Holds the previous postion of the player. Global for use in functions
past_x = 1
past_y = 1

#print(past_x, past_y)

# Creates a 2D array to store grid
#grid_size = input("Grid size please? /n")
grid_size = 16
maze = [[" + " for i in range(grid_size+1)] for i in range(grid_size+1)]

#Function for printing grid. Concatonates the elements from each row
def maze_print(maze):
    for i in range(len(maze)-1):
        line = ""
        for j in range(len(maze[i])-1):
            line += maze[i][j]
        print(line)
#print(maze)

#Creates n number random obstacles and puts them in grid
num_blocks  = 20
while num_blocks:
    num_blocks -= 1
    maze[random.randint(1, grid_size)][random.randint(1, grid_size)] = "▉"

# Updates grid to place postion of current
def update_maze(x_pos, y_pos):
    # Clears past grid
    output.clear()

    global past_x
    global past_y

    # Puts empty tile back where user was last turn (if here was a last turn)
    if past_x != None and past_y != None:
        maze[past_x][past_y] = " + "
    # Puts player tile on current postion
    maze[x_pos][y_pos] = " 🎉 "

    # Sets past position to current position
    past_x = x_pos
    past_y = y_pos

    maze_print(maze)

    # Some experimental code that doesn't quite work yet. Rather than a maze, the game is to try to remove all the blocks
    """
    # Removes block if player is on it
    if maze[x_pos][y_pos] == "▉":
        num_blocks -= 1

        # Win message if all blocks are gone
        if num_blocks == 0:
            print("You Won!")

        #Updates and prints block
        else:
    """
#Testing output
update_maze(4, 3)

 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  + ▉ +  +  +  + ▉ +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  + ▉ +  +  +  +  +  +  +  + 
 +  +  +  🎉  +  +  +  +  +  +  +  +  +  +  +  + 
 + ▉ +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + ▉ +  +  + 
 +  +  +  +  + ▉ +  +  +  +  +  +  +  +  + ▉
 +  +  + ▉ +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 + ▉ +  +  +  +  +  +  + ▉ +  +  + ▉▉ + 
 +  +  +  +  +  +  +  +  +  +  +  +  + ▉ +  + 
 +  +  + ▉ +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  + ▉ +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  +  + ▉ +  + 
 +  +  +  +  +  +  +  +  +  + ▉ +  +  +  +  + 


In [ ]:
# JS Camera and meidapipe operations per frame

#Calling function for video feed
video_stream()

#Function call shorthand
mp_hands = mp.solutions.hands

while True:
    #1 sec wait to make it less jumpy
    time.sleep(2)

    unprocessed_frame = video_frame("Capturing", '')
    #Will stop running if feed is closed
    if not unprocessed_frame:
        print("Program has been closed. Have a nice day!")
        break


    #Processing frame:
    # Takes js feed and makes it a BGR image
    current_frame = js_to_image(unprocessed_frame["img"])
    #frame = cv2.flip(current_frame, 1) # Horizontal flip
    #print(current_frame)


    #meidapipe stuff (hand tracking):
    # Sets parameters of hand tracking model to process frames as video, detect 1 hand, with a moderate amount of confidance
    with mp_hands.Hands(
        static_image_mode = False,
        max_num_hands = 1,
        min_detection_confidence = 0.8
            ) as hands:

        # .process Takes an RGB image (BGR2RGB converts it) and creates hand "landmakrs" for it
        frame = hands.process(cv2.flip(cv2.cvtColor(current_frame, cv2.COLOR_BGR2RGB), 1))

        # Restarts loop if no landmarks detected
        if not frame.multi_hand_landmarks:
            continue

        """
        # Will only get x & y of right hand index finger
        if frame.multi_handedness == "Right":
        """
        # Looks throgh the hand landmarks and looks for an index finger
        # Finds the x and y coordinates of it
        for hand_landmarks in frame.multi_hand_landmarks:
            x_cood = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * grid_size)
            y_cood = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * grid_size)
            #print(x_cood, y_cood)

            #Takes those x and y coordinates and sends them off to the update_maze function
            update_maze(y_cood, x_cood)


 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  + ▉ +  +  +  + ▉ +  +  +  +  + 
 +  +  +  +  +  +  +  +  🎉  +  +  +  +  +  +  + 
 +  +  +  +  +  +  + ▉ +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 + ▉ +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + ▉ +  +  + 
 +  +  +  +  + ▉ +  +  +  +  +  +  +  +  + ▉
 +  +  + ▉ +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  + 
 + ▉ +  +  +  +  +  +  + ▉ +  +  + ▉▉ + 
 +  +  +  +  +  +  +  +  +  +  +  +  + ▉ +  + 
 +  +  + ▉ +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  + ▉ +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  +  + ▉ +  + 
 +  +  +  +  +  +  +  +  +  + ▉ +  +  +  +  + 


KeyboardInterrupt: 